In [ ]:
%matplotlib inline


# Anonymize DICOM data


This example is a starting point to anonymize DICOM data.

It shows how to read data and replace tags: person names, patient id,
optionally remove curves and private tags, and write the results in a new file.


In [ ]:
import pydicom

## Anonymize a single file

In [ ]:
filename = '../DATASETS/IMAGES/DICOM_Examples/Brain_MRI/IM67_1slice.dcm'
dataset = pydicom.dcmread(filename)

In [ ]:
elements = ['PatientID',
                 'PatientBirthDate']
for element in elements:
    print(dataset.data_element(element))

In [ ]:
dataset

We can define a callback function to find all tags corresponding to a person
names inside the dataset.

In [ ]:
def person_names_callback(dataset, data_element):
    if data_element.VR == "PN": #VR = value representation, PN=persons's name
        data_element.value = "anonymous"
        
def scanner_info_callback(dataset, data_element):
    if data_element.VR == "LO":
        data_element.value = "scanner info"

We can use the different callback function to iterate through the dataset but
also some other tags such that patient ID, etc.

This can be achieved by means of the `walk` method, which iterates over the data elements of the *FileDataset* object:

In [ ]:
dataset.walk(person_names_callback)
dataset.walk(scanner_info_callback)

or, equivalently, as:

In [ ]:
callbacks = [person_names_callback, scanner_info_callback]
for callback in callbacks:
    dataset.walk(callback)

pydicom allows to remove private tags using `remove_private_tags` method

In [ ]:
dataset.remove_private_tags()

Optional data elements can be easily deleted using `del` or `delattr`.



In [ ]:
if 'OtherPatientIDs' in dataset:
    delattr(dataset, 'OtherPatientIDs')

if 'OtherPatientIDsSequence' in dataset:
    del dataset.OtherPatientIDsSequence

Data can also be modified via assignments:

In [ ]:
dataset.OperatorsName= 'Lucio Verdi'

# Anonymizing the birthdate

Let's try to set the birth date to the first day of the month

In [ ]:
import datetime
import time

In [ ]:
date = '20000122'

In [ ]:
format_ = "%Y%m%d"
time_struct = time.strptime(date, format_)
time_struct

In [ ]:
birth_date = datetime.datetime(*time_struct[:3])
birth_date

datetime.datetime objects are immutable

In [ ]:
birth_date.day = 1

In [ ]:
new_date = birth_date.replace(day=1, month=5)
new_date

In [ ]:
new_date.strftime(format_)

In [ ]:
def anonimize_day(date_str, format_="%Y%m%d", day=1):
    time_struct = time.strptime(date_str, format_)
    date = datetime.datetime(*time_struct[:3])
    new_date = date.replace(day=day)
    return new_date.strftime(format_)


In [ ]:
tag = 'PatientBirthDate'
if tag in dataset:
    date_str = dataset.data_element(tag).value
    dataset.data_element(tag).value = anonimize_day(date_str, day=5)
dataset.PatientBirthDate

Finally, it is possible to store the image



In [ ]:
output_filename ='IM67_orig_anon.dcm'
dataset.save_as(output_filename)

## Anonymize a folder

In [ ]:
import glob
import os

In [ ]:
DIR_NAME = '../DATASETS/IMAGES/DICOM_Examples/Breast_Mammography_Case2/'
PATTERN='*.dcm'

In [ ]:
!ls -R '../DATASETS/IMAGES/DICOM_Examples/Breast_Mammography_Case2/'

Here we define a generator to get the names of the dicom files and an anonymization function

In [ ]:
def file_list_generator(dir_name, pattern):
    for path, subfolder, files in os.walk(dir_name):
        for file in files:
            if glob.fnmatch.fnmatch(file, pattern):
                yield os.path.join(path, file)

In [ ]:
def anonymize_file(fname):
    dataset = pydicom.dcmread(fname)
    callbacks = [person_names_callback, scanner_info_callback]
    for callback in callbacks:
        dataset.walk(callback)
    output_filename = fname.replace('.dcm', '_anonym.dcm')
    dataset.save_as(output_filename)


In [ ]:
import multiprocessing
pool = multiprocessing.Pool()

The map operator applies the function *anonimyze_file* to each element of the iterator *file_list_generator*.

In [ ]:
pool.map(anonymize_file, file_list_generator(DIR_NAME, PATTERN))

In [ ]:
!ls -R '../DATASETS/IMAGES/DICOM_Examples/Breast_Mammography_Case2/'

In [ ]:
#Cleaning up residual files
!find '../DATASETS/IMAGES/DICOM_Examples/Breast_Mammography_Case2/' -name *_anonym.dcm -exec rm {} \;